In [36]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.model_selection
#from sklearn.model_selection import cross_validate
#做 split 要加上
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
data = pd.read_csv("data.csv",sep=",")
print(data.shape)
print data.info()

(41188, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
ind               41188 non-null int64
age               41188 non-null int64
job               40858 non-null object
marital           41108 non-null object
education         39457 non-null object
default           32591 non-null object
housing           40198 non-null object
loan              40198 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          5625 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
d

In [3]:
# drop observations with missing data
data["poutcome"]=data["poutcome"].fillna("nonexistent") # most value of "poutcome" are missing, keep them
data=data.dropna(how='any')
print data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30488 entries, 0 to 41187
Data columns (total 22 columns):
ind               30488 non-null int64
age               30488 non-null int64
job               30488 non-null object
marital           30488 non-null object
education         30488 non-null object
default           30488 non-null object
housing           30488 non-null object
loan              30488 non-null object
contact           30488 non-null object
month             30488 non-null object
day_of_week       30488 non-null object
duration          30488 non-null int64
campaign          30488 non-null int64
pdays             30488 non-null int64
previous          30488 non-null int64
poutcome          30488 non-null object
emp.var.rate      30488 non-null float64
cons.price.idx    30488 non-null float64
cons.conf.idx     30488 non-null float64
euribor3m         30488 non-null float64
nr.employed       30488 non-null float64
y                 30488 non-null object
dtypes: floa

In [4]:
#Pre Condition: One Hot encode
def one_hot_encode(data,column_name,index_name):
    n = data.shape[0] #length of columns 
    binary_data = pd.DataFrame() #new dataframe to store cleaned data
    old_row = data[column_name]
    UniqueVal=[]
    for elem in old_row:       
        if not elem in UniqueVal:
            UniqueVal.append(elem)
    #creates new columns out of entries of old column
    for new_col in UniqueVal:        
        if new_col=="nonexistent":
            #print ("poutcome."+new_col)
            binary_data["poutcome."+new_col] = (old_row==new_col)+0.0
        else:
            binary_data[new_col] = (old_row==new_col)+0.0
    binary_data[index_name]=data[index_name]
    return binary_data

def binary_encode(data, column_name,index_name): # name "yes" as "1"
    n = data.shape[0] #length of columns 
    binary_data = pd.DataFrame() #new dataframe to store cleaned data
    old_row = data[column_name]
    UniqueVal=["yes","no"]
    #creates new columns out of entries of old column
    binary_data[column_name] = (old_row=="yes")+0.0
    binary_data[index_name]=data[index_name]
    return binary_data

In [5]:
#clean data
categorical_multi = ["job","marital","education","contact","month","day_of_week","poutcome"]
categorical_binary=["default","housing","loan","y"]
categorical_num=["ind","age","duration","campaign","previous","emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]#,"pdays"
flag=0
for col in categorical_multi:
    if flag==0:
        clean_data= one_hot_encode(data,col,"ind")
        flag+=1        
    clean_data=pd.merge(clean_data,one_hot_encode(data,col,"ind"),how='left')
    
for col in categorical_binary:
    clean_data=pd.merge(clean_data,binary_encode(data,col,"ind"),how='left')

    
# n = data.shape[0] #length of columns 
# sep_pdays_data = pd.DataFrame() #new dataframe to store cleaned data
# sep_pdays_data[index_name]=data[index_name]
# old_row = data["pdays"]
# #creates new columns out of entries of old column
# sep_pdays_data["newcustomer"] = (old_row==999)+0.0
# sep_pdays_data["pdays"]=sep_pdays_data["pdays"]
                    
clean_data=pd.merge(clean_data,data[categorical_num])
clean_data=clean_data.drop("ind",axis=1)





In [6]:
## data for tree models(without normalization)
data_DT=clean_data

## divide dataset into traininng(0.8),testing set(0.2)

testing_set,training_set= train_test_split(data_DT, test_size=0.8)
y_testing_DT=testing_set["y"]
x_testing_DT=testing_set.drop("y",axis=1)
y_training_DT=training_set["y"]
x_training_DT=training_set.drop("y",axis=1)
#drop duration
x_training_DT=x_training_DT.drop("duration",axis=1)
x_testing_DT=x_testing_DT.drop("duration",axis=1)
print x_training_DT.shape

(24391, 52)


In [7]:
# normalization
categorical_num=["age","duration","campaign","previous","emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]#pdays
for elem in categorical_num:
    #clean_data[elem].apply(lambda x:(x-np.mean(x))/np.std(x))
    clean_data[elem]=(clean_data[elem]-clean_data[elem].mean())/clean_data[elem].std()

In [8]:
# Return cleaned data 
clean_data.to_csv("clean_data.csv", index=False)

In [9]:
## divide dataset into traininng(0.8),testing set(0.2)

testing_set,training_set= train_test_split(clean_data, test_size=0.8)
#validating_set,training_set = train_test_split(tmp, test_size=0.75)
y_testing=testing_set["y"]
x_testing=testing_set.drop("y",axis=1)
# y_validating=validating_set["y"]
# x_validating=validating_set.drop("y",axis=1)
y_training=training_set["y"]
x_training=training_set.drop("y",axis=1)
#drop duration
x_training=x_training.drop("duration",axis=1)
#x_validating=x_validating.drop("duration",axis=1)
x_testing=x_testing.drop("duration",axis=1)
print x_training.shape
# count=sum((x_training["y"]==1)+0.0)
# print count
x_training

(24391, 52)


,housemaid,services,admin.,technician,blue-collar,unemployed,retired,entrepreneur,management,student,...,housing,loan,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
29877,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,-0.002904,-0.559326,-0.371610,-1.011233,0.860456,0.480715,-1.441533,-2.251434
18117,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.255136,0.543554,-0.371610,-0.017691,-0.552315,-0.291849,0.346642,0.465507
17015,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,-0.970628,-0.191699,-0.371610,-0.017691,-0.552315,-0.291849,0.389968,0.465507
1634,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.577730,0.911181,-0.371610,0.727466,0.804082,0.877437,0.785527,0.401641
6589,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,-0.196449,0.175927,-0.371610,0.913755,1.608695,-0.250089,0.844045,0.895268
20782,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.583538,-0.559326,-0.371610,-1.073330,-0.765854,-1.356734,-1.156258,-0.821115
26957,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.196449,0.175927,3.454033,-1.756390,-2.258916,1.921442,-1.449411,-1.125806
6300,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.293221,5.322702,-0.371610,0.913755,1.608695,-0.250089,0.843482,0.895268
18125,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,-0.970628,-0.559326,1.541211,-0.017691,-0.552315,-0.291849,0.346642,0.465507
3696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-0.777083,-0.191699,-0.371610,0.727466,0.804082,0.877437,0.786089,0.401641


In [10]:
# Fit lasso regression model--feature selection
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
model_select=np.zeros((20,2))
tmp=0
lamda=0

for i in np.linspace(1,20,20):
    clf = linear_model.Lasso(alpha=i/10.0)#0.1-2.0
    model_select[int(i-1),0]=i/10.0
    model_select[int(i-1),1]=np.mean(cross_val_score(clf,x_training,y_training,cv=10))
    if model_select[int(i-1),1]>tmp:
        tmp= model_select[int(i-1),1]
        lamda=i/10.0
    
print "Best_Lasso:\nLambda=",lamda,"\naccuracy=",tmp


Best_Lasso:
Lambda= 0.1 
accuracy= 0.0421884445936


In [15]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_training,y_training)
# Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
#    normalize=False, positive=False, precompute=False, random_state=None,
#    selection='cyclic', tol=0.0001, warm_start=False)
print(clf.coef_)

[ 0.         -0.          0.         -0.         -0.          0.          0.
 -0.         -0.          0.         -0.         -0.          0.         -0.
  0.         -0.         -0.         -0.         -0.          0.          0.
 -0.          0.         -0.          0.         -0.         -0.          0.
 -0.          0.          0.          0.          0.         -0.          0.
  0.          0.         -0.         -0.          0.          0.         -0.
  0.         -0.          0.         -0.          0.         -0.         -0.
  0.         -0.         -0.02153499]


#### Lasso Model is not suitable to be applied on this dataset

# Fit logistic regression model

In [11]:
# Fit logistic regression model
model = LogisticRegression(penalty='l1', C=1, multi_class='ovr')
model.fit(x_training, y_training)
print("Training accuracy: ", model.score(x_training, y_training))
#print("Cross validation accuracy: ", model.score(x_training, x_training))
#model.score(x_testing, y_testing)
model_select=np.zeros((20,2))
tmp=0
lamda=0
for i in np.linspace(1,20,20):
    clf = LogisticRegression(penalty='l1', C=i/10.0, multi_class='ovr')#0.1-2.0
    model_select[int(i-1),0]=i/10.0
    model_select[int(i-1),1]=np.mean(cross_val_score(clf,x_training,y_training,cv=10))
    if model_select[int(i-1),1]>tmp:
        tmp= model_select[int(i-1),1]
        lamda=i/10.0
print "Best_Logistic:\nLambda=",lamda,"\naccuracy=",tmp
score(X, y, sample_weight=None)

('Training accuracy: ', 0.88778647861916282)
Best_Logistic:
Lambda= 0.1 
accuracy= 0.887458424482


In [33]:
#  Logistic Regression Model (find unefficient features)
model = LogisticRegression(penalty='l1', C=0.1, multi_class='ovr')
model.fit(x_training, y_training)
print "Coefficients:", model.coef_
print x_training.columns.values
#print("Testing accuracy: ", model.score(x_testing, y_testing))

Coefficients: [[ 0.         -0.12335421  0.00543509  0.         -0.14532764  0.
   0.16022173  0.          0.          0.11516578  0.         -0.02527353
   0.         -0.02333491 -0.01359553  0.          0.          0.          0.
   0.05200062  0.         -0.63516874  0.         -0.5810577  -0.27313616
   0.          0.03518305  0.         -0.39658506  0.          0.9717043
   0.         -0.09230158 -0.32026454  0.          0.07493433  0.
  -0.12437032 -0.95906464 -1.43608555  0.26066877  0.         -0.03596629
   0.          0.         -0.11496395  0.01425905 -1.21837461  0.66550766
   0.12866684  0.          0.        ]]
['housemaid' 'services' 'admin.' 'technician' 'blue-collar' 'unemployed'
 'retired' 'entrepreneur' 'management' 'student' 'self-employed' 'married'
 'single' 'divorced' 'basic.4y' 'high.school' 'basic.6y'
 'professional.course' 'basic.9y' 'university.degree' 'illiterate'
 'telephone' 'cellular' 'may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar'
 'apr' 'sep' 'mon' 'tue

According to the result from best Logistic Model, some features are of low correlation with classification. Thus, we prefer to remove them form the model, Such as 'euribor3m', 'nr.employed' and so on. Though the coefficents of some of dummy variables (first 41 features in the list) are also 0, we cannot remove them, because they are generated by one-hot encoding.

In [32]:
# Test Logistic Regression Model (drop 'euribor3m', 'nr.employed',...)
x_training_ed=x_training.drop(["euribor3m","nr.employed","loan", 'age','default'],axis=1)
model = LogisticRegression(penalty='l1', C=0.1, multi_class='ovr')
model.fit(x_training_ed, y_training)
print "Training Accuracy:", model.score(x_training_ed, y_training)
print "Coefficients:", model.coef_
print x_training_ed.columns.values
# test the model
x_testing_ed=x_testing.drop(["euribor3m","nr.employed","loan", 'age','default'],axis=1)
print [model.predict(x_testing_ed) y_testing]
print("Testing accuracy: ", model.score(x_testing_ed, y_testing))

Training Accuracy: 0.887950473535
Coefficients: [[ 0.         -0.12366751  0.00538798  0.         -0.14561966  0.
   0.16007952  0.          0.          0.11482946  0.         -0.02561143
   0.         -0.02350662 -0.01353623  0.          0.          0.          0.
   0.05173809  0.         -0.63532375  0.         -0.58086062 -0.2732672
   0.          0.03523139  0.         -0.396425    0.          0.97167669
   0.         -0.09257061 -0.32035795  0.          0.07477912  0.
  -0.12456011 -0.66301783 -1.13952995  0.55737815 -0.03618094 -0.11493228
   0.01406197 -1.21850775  0.66575119  0.12872286]]
['housemaid' 'services' 'admin.' 'technician' 'blue-collar' 'unemployed'
 'retired' 'entrepreneur' 'management' 'student' 'self-employed' 'married'
 'single' 'divorced' 'basic.4y' 'high.school' 'basic.6y'
 'professional.course' 'basic.9y' 'university.degree' 'illiterate'
 'telephone' 'cellular' 'may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar'
 'apr' 'sep' 'mon' 'tue' 'wed' 'thu' 'fri' 'poutcom

Though we discard some of the features and fit the model again, the accuracy of the model is stable (testing accuracy is 0.89), which justifies our revision.

## SVM

In [41]:
# SVM cross-validation
from sklearn import svm
model_select=np.zeros((5,2))
tmp=0
lamda=0
for i in np.linspace(1,5,5):
    clf = svm.SVC(kernel='linear', C=i*0.6)#choose penalty parameter within 0.6-3.0
    model_select[int(i-1),0]=i*0.6
    model_select[int(i-1),1]=np.mean(cross_val_score(clf,x_training,y_training,cv=4))
    if model_select[int(i-1),1]>tmp:
        tmp= model_select[int(i-1),1]
        lamda=i*0.6
print "Best_SVM:\nLambda=",lamda,"\naccuracy=",tmp

Best_SVM:
Lambda= 0.6 
accuracy= 0.88483463847


In [43]:
# Test SVM Model
model = svm.SVC(kernel='linear', C=0.6)
model.fit(x_training, y_training)
print("Testing accuracy: ", model.score(x_testing, y_testing))

('Testing accuracy: ', 0.88797769394784321)


## single decision tree

In [415]:
# Fit a single decision tree
model_select=np.zeros((16,2))
tmp=0
lamda=0

print("Training accuracy: ", model.score(x_training, y_training))
for i in np.linspace(5,20,16):#check max depth from 5-20
    model = DecisionTreeClassifier(max_features=None, max_depth=int(i))#0.1-2.0
    model.fit(x_training_DT, y_training_DT)
    model_select[int(i-5),0]=int(i)
    model_select[int(i-5),1]=model.score(x_training_DT, y_training_DT)
    if model_select[int(i-5),1]>tmp:
        tmp= model_select[int(i-5),1]
        max_depth=int(i)
print "Best_DT:\nmax_depth=",max_depth,"\naccuracy=",tmp

('Training accuracy: ', 0.9338690500594482)
Best_DT:
max_depth= 20 
accuracy= 0.967078020581


## plot decision tree

In [61]:
model_DT = DecisionTreeClassifier(max_features=None, max_depth=5)
model_DT.fit(x_training_DT, y_training_DT)
print("Training accuracy: ", model_DT.score(x_training_DT, y_training_DT))
print("Testing accuracy: ", model_DT.score(x_testing_DT, y_testing_DT))
from IPython.display import Image 
import graphviz 
from sklearn import tree

# dot_data = tree.export_graphviz(model, out_file='tree.dot') 
# graph = graphviz.Source(dot_data) 
Feature_names=['housemaid','services','admin.','technician','blue-collar','unemployed','retired','entrepreneur','management','student','self-employed','married','single','divorced','basic.4y','high.school','basic.6y'
,'professional.course','basic.9y','university.degree','illiterate','telephone','cellular','may','jun','jul','aug','oct','nov','dec','mar','apr','sep','mon','tue','wed','thu','fri','poutcome.nonexistent','failure'
,'success','default','housing','loan','age','campaign','previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']
Class_names=['0','1']
dot_data = tree.export_graphviz(model_DT, out_file=None,feature_names=Feature_names,class_names=Class_names,filled=True, rounded=True,special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

('Training accuracy: ', 0.89229633881349679)
('Testing accuracy: ', 0.88961784484172546)


ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

In [223]:
base_model = DecisionTreeClassifier(max_features=None, max_depth=20)
#base_model = LogisticRegression(penalty='l2', C=10, multi_class='ovr')
model = BaggingClassifier(base_estimator=base_model, n_estimators=50)
model.fit(x_training, y_training)
print("Training accuracy: ", model.score(x_training, y_training))

('Training accuracy: ', 0.99967202361429974)


In [64]:
# Fit random forest
modelrf = RandomForestClassifier(n_estimators=100, max_depth=30, max_features='auto')
modelrf.fit(x_training_DT, y_training_DT)
print("Training accuracy: ", modelrf.score(x_training_DT, y_training_DT))
print("Testing accuracy: ", modelrf.score(x_testing_DT, y_testing_DT))

('Training accuracy: ', 0.99364519699889309)
('Testing accuracy: ', 0.87698868295883226)


In [66]:
# Fit boosted model
base_model = DecisionTreeClassifier(max_features='auto', max_depth=30)
#base_model = LogisticRegression(penalty='l2', C=10, multi_class='ovr')
model = AdaBoostClassifier(base_estimator=base_model, n_estimators=10)
model.fit(x_training_DT, y_training_DT)
print("Training accuracy: ", model.score(x_training_DT, y_training_DT))
print("Testing accuracy: ", model.score(x_testing_DT, y_testing_DT))

('Training accuracy: ', 0.99401418556024768)
('Testing accuracy: ', 0.84812202722650487)
